## Classificador Bayesiano - AI Component - Classification Lyrics

### Importar Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, CategoricalNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import joblib

### Referencia do CountVectorizer - Bag of Words

In [ ]:
letras = ['''
Eu sei que vou te amar
Por toda a minha vida eu vou te amar
Em cada despedida eu vou te amar
Desesperadamente, eu sei que vou te amar
E cada verso meu será
Pra te dizer que eu sei que vou te amar
Por toda minha vida
Eu sei que vou chorar
A cada ausência tua eu vou chorar
Mas cada volta tua há de apagar
O que esta ausência tua me causou
Eu sei que vou sofrer a eterna desventura de viver
A espera de viver ao lado teu
Por toda a minha vida
''']

vect = CountVectorizer()
X = vect.fit_transform(letras)

df = pd.DataFrame(X.toarray(), columns=vect.get_feature_names_out())

In [ ]:
df

### Leitura do DataSet

In [ ]:
df_lyrics = pd.read_excel("dataset_genero_musical.xlsx")

In [ ]:
df_lyrics.head(5)

In [ ]:
df_lyrics.dropna(inplace=True)

In [ ]:
print(df_lyrics["musica"].values[0])

### Limpeza de Dados

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
list_stops_words = stopwords.words("portuguese")
list_stops_words

In [ ]:
# str.replace(r'[^\w\s]', '', regex=True):
#  remove tudo que não é caractere de palavra (\w = letras, dígitos, _, com suporte Unicode)
#nem espaço (\s). Na prática: tira pontuação/símbolos.

#str.replace('\n', ' ', regex=True): troca quebras de linha por espaço para não “grudar” palavras.

#str.lower(): converte tudo para minúsculas.

df_lyrics['musica'] = (
    df_lyrics['musica']
      .str.replace(r'[^\w\s]', '', regex=True)
      .str.replace('\n', ' ', regex=True)
      .str.lower()
)

In [ ]:
print(df_lyrics["musica"].values[0])

### Separação em Treinamento e Teste - Hold-Out

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_lyrics["musica"], df_lyrics["genero"],
                                                    test_size=0.30, #Usando 30% de amostras pra teste
                                                    random_state=42) # Semente aleatória nas amostras

### Pipeline Model

In [ ]:
classify = Pipeline(
                [('vect', CountVectorizer(stop_words= list_stops_words, ngram_range=(1, 2))), #1 - Extração de Características Numéricas
                 ('clf', MultinomialNB()), #2- Classificador
                 ])

#### Ajuste do Modelo

In [ ]:
classify.fit(X_train, y_train)

### Avaliação do Modelo

Leitura sobre Métricas de Avaliação

https://medium.com/@mateuspdua/machine-learning-m%C3%A9tricas-de-avalia%C3%A7%C3%A3o-acur%C3%A1cia-precis%C3%A3o-e-recall-d44c72307959

In [ ]:
preds = classify.predict(X_test)

print(metrics.classification_report(y_test, preds))

### Predição do Modelo

In [ ]:
letra  = '''

Eu sei que vou te amar
Por toda a minha vida eu vou te amar
Em cada despedida eu vou te amar
Desesperadamente, eu sei que vou te amar
E cada verso meu será
Pra te dizer que eu sei que vou te amar
Por toda minha vida
Eu sei que vou chorar
A cada ausência tua eu vou chorar
Mas cada volta tua há de apagar
O que esta ausência tua me causou
Eu sei que vou sofrer a eterna desventura de viver
A espera de viver ao lado teu
Por toda a minha vida

'''

In [ ]:
classify.predict([letra])

In [ ]:
classify.classes_

In [ ]:
classify.predict_proba([letra])

### Exportar Modelo - Componente

In [ ]:
file_name = 'ai_component_cla_lyrics.unisagrado'
joblib.dump(classify, file_name)

### Leitura do Modelo - Componente

In [ ]:
component_name = 'ai_component_cla_lyrics.unisagrado'

component_ai = joblib.load(component_name)

In [ ]:
component_ai.predict([letra])

In [ ]:
component_ai.classes_

In [ ]:
component_ai.predict_proba([letra])